In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pandarallel import pandarallel

In [2]:
# -----------------------------CARGA DE DATOS----------------------------
d = pd.read_parquet('./datasets/taxi_trip/8_months_taxi.parquet')
#d2 = pd.read_parquet('./datasets/taxi_trip/yellow_tripdata_2017-12.parquet')
#d = pd.concat([d1, d2])
#d.reset_index(inplace=True)

In [3]:
# --------------------------ELIMINACIOND E CELDA--------------------
#df.drop(columns=['Unnamed: 0','datetime'], inplace=True)

In [4]:
#-------------------------MODIFICACION DE CELDA-------------------
#df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
#df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df = pd.DataFrame(pd.to_datetime(d.tpep_pickup_datetime))

CAMBIO DE METODO DE OBTENCION DE MASCARAS

In [5]:
#-----------------------------CELDA AGREGADA---------------
def getDay(param):
    return str(param).split(':')[0]

def getHour(param):
    return int(param.strftime("%H"))

In [6]:
pandarallel.initialize()
df['dia_hora'] = df.tpep_pickup_datetime.parallel_apply(getDay)
df['hora'] = df.tpep_pickup_datetime.parallel_apply(getHour)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [7]:
# Dias
#mask_dia = np.zeros((df.tpep_pickup_datetime.shape[0], 1))
#for idx, i in enumerate(df.tpep_pickup_datetime):
#    mask_dia[idx] =  str(i.strftime("%d"))#+"-"+i.strftime("%H")
#    
#mask_dia = mask_dia.sum(axis=1)
#df['dia_hora'] = mask_dia
#df.dia_hora = df.dia_hora.astype('str')
#
## Horas
#mask_hora = np.zeros((df.tpep_pickup_datetime.shape[0], 1))
#for idx, i in enumerate(df.tpep_pickup_datetime):
#    mask_hora[idx] =  i.strftime("%H")
#    
#mask_hora = mask_hora.sum(axis=1)
#df['hora'] = mask_hora
#df['dia'] = mask_dia
#df.hora = df.hora.astype('int')
#
##horas_dia
#df.dia_hora = df.dia_hora + '-' + df.hora.astype('str')

In [8]:
d = df.groupby(['dia_hora','hora']).size()
statistics = pd.DataFrame(d.values, columns=['viajes_amount'])


In [9]:
# get index from statistics
horas = np.zeros((d.values.shape[0],1))
for idx, i in enumerate(d.index):
    horas[idx] = i[1]

statistics['hora'] = horas

In [10]:
##-------------------------------ELIMINACION DE BLOQUE------------------------

#f = statistics.hora.unique()
#
#boxes = list(range(0,24))
##h = f[[3, 1, 5, 6, 4, 0, 2]]
#
#for i in boxes:
#    boxes[i] = np.array(statistics.viajes_amount[statistics.hora == f[i]])

In [14]:
#---------------------------MULTIPLICAR POR 8 PARA COMPENZAR LA CANTIDAD DEL SAMPLE--------------
f = statistics.hora.unique()
f = np.sort(f)
boxes = list(range(0,24))
#h = f[[3, 1, 5, 6, 4, 0, 2]]

for i in boxes:
    boxes[i] = np.array(statistics.viajes_amount[statistics.hora == f[i]]*8)

In [15]:
f = list(range(0,24))

In [16]:
fig = go.Figure()
for idx, i in enumerate(boxes):
    fig.add_trace(go.Box(y=i, name=f[idx],
                marker_color = 'blue'))

fig.update_layout(
    autosize=False,
    width=800,
    height=500,
    font_color="blue",
    title = 'Distribucion de viajes por hora',
    #title_font_family="Times New Roman",
    title_font_color="black",
    title_font_size = 25
    )

fig.show()